I decided to take a look into meteostat (https://github.com/meteostat/meteostat-python) as it is the library we will use as data source for our project. First I need to install it.





In [7]:
pip install meteostat folium

Then some basic functions from other libraries (pyplot for a simple plot, dateime for a proper date format) as well as the meteostat functions are importet

In [8]:
from datetime import datetime,timedelta,date
import matplotlib.pyplot as plt
from meteostat import Daily, Point, Normals, Monthly, Stations
import numpy
from geopy.distance import geodesic
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
import folium
import pandas as pd
import math

In [3]:
#Get the 100 closest stations to Salzburg, drop those without data and add the distance to Salzburg as additional parameter.
#This can be used for forcasting methods (as likely stations further away have less influence on the waether in Salzburg)
coord=(47.792,13.0477)
day=datetime(2024,5,20)

stations = Stations()
stations = stations.nearby(coord[0],coord[1])
stationSbg2 = stations.fetch(100)

test=stationSbg2.dropna()
indexes=test.index

for i in indexes:
  data=Daily(str(i),day-timedelta(10000),day)
  data=data.fetch()
  stat=stationSbg2.loc[str(i)]
  locatio=(stat['latitude'],stat['longitude'])
  distance=geodesic(coord,locatio).km
  #print(distance)
  data=data.assign(distance=distance)
  #print(i)
  print(data)

            tavg  tmin  tmax  prcp  snow   wdir  wspd  wpgt    pres  tsun  \
time                                                                        
1997-01-02 -10.6 -15.3  -6.9   0.0  51.0    NaN   3.0   NaN  1022.2   NaN   
1997-01-03  -6.9  -9.4  -2.4   0.0  51.0    NaN   NaN   NaN     NaN   NaN   
1997-01-04  -4.7 -10.8  -2.1   1.0  51.0    NaN   NaN   NaN     NaN   NaN   
1997-01-05  -1.3  -2.9   0.3   0.0  79.0    NaN   4.2   NaN  1010.6   NaN   
1997-01-06  -1.6  -1.8  -0.3   0.0  61.0  314.0   5.8   NaN  1017.5   NaN   
...          ...   ...   ...   ...   ...    ...   ...   ...     ...   ...   
2024-05-16  19.2  16.7  22.3   0.0   NaN  117.0  17.7  47.0  1003.8   NaN   
2024-05-17  13.7  11.2  16.9   3.1   NaN    2.0  11.7  38.9  1007.2   NaN   
2024-05-18  16.0  10.1  21.9   0.9   NaN   29.0  11.1  31.3  1011.2   NaN   
2024-05-19  16.7  11.2  22.0   1.2   NaN    6.0  11.7  55.4  1011.4   NaN   
2024-05-20  18.2  11.8  23.9   3.0   NaN   40.0  13.1  37.1  1009.6   NaN   

In [3]:
def nearest_station(x,y,year):
  stations = Stations()
  stations = stations.nearby(x,y)
  testi = stations.fetch()
  for i in testi.index:
      if math.isnan(Normals(i,year,year+29).fetch()['tavg'].mean())==False:
        a=i
        break
  return a

def temp_change(x,y,tstart):
  tend=tstart+29
  data3=Normals(nearest_station(x,y,tstart),tstart,tend)
  data3=data3.fetch()
  startdate=datetime.combine(date.today(), datetime.min.time())-timedelta(394)
  data4=Monthly(nearest_station(x,y,tstart),startdate,datetime.combine(date.today(), datetime.min.time())-timedelta(30))
  data4=data4.fetch()
  data5=data4['tavg'].mean()-data3['tavg'].mean()
  return data5

def gridapply(grid,tstart):
  grid['station_id']=0
  grid['tempchange']=-999
  for i in grid.index:
    grid['station_id'].iloc[i]=nearest_station(grid['latitude'].iloc[i],grid['longitude'].iloc[i],tstart)
    grid['tempchange'].iloc[i]=temp_change(grid['latitude'].iloc[i],grid['longitude'].iloc[i],tstart)
  return grid

math.isnan(Normals('65467',1961,1961+29).fetch()['tavg'].mean())
temp_change(49.6,11.0,1991)
#nearest_station(49.6,11,1961)
#Normals(nearest_station(49.6,11,1961),1961,1990).fetch()


1.1386363636363654

In [5]:
long=numpy.arange(-180,180,1)
long=numpy.repeat(long,len(long))
lat=numpy.arange(-180,180,1)
lat=numpy.tile(lat,len(lat))
grid=pd.DataFrame()
grid['latitude']=lat
grid['longitude']=long
grid['station_id']=0
grid['tempchange']=-999
for i in grid.index:
  grid['station_id'].iloc[i]=nearest_station(grid['latitude'].iloc[i],grid['longitude'].iloc[i],1961)
  grid['tempchange'].iloc[i]=temp_change(grid['latitude'].iloc[i],grid['longitude'].iloc[i],1961)
grid


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


KeyboardInterrupt: 

In [6]:
long=numpy.arange(-180,180,1)
long=numpy.repeat(long,len(long))
lat=numpy.arange(-180,180,1)
lat=numpy.tile(lat,len(lat))
grid=pd.DataFrame()
grid['latitude']=lat
grid['longitude']=long
gridapply(grid,1961)

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


KeyboardInterrupt: 